# reg_part_B_NN

# Data import libraries

In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import torch
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge

In [22]:
name_data_file="heart_failure_clinical_records_dataset"

data = pd.read_csv(f"../../raw_data/{name_data_file}.csv", na_values=["?"])

In [3]:
data

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


# Cross validation pipeline

In [152]:
X = data.drop(columns=['time'])
y = data['time']   # pandas Series

N, M = X.shape

# X.shape, y.shape print shapes of X and y to undestand their dimensions

## Help Functions

In [145]:
# Normalize data based on training set

def get_fold_data(X, y, train_idx, val_idx):
   
    X_train = X.iloc[train_idx]
    X_val   = X.iloc[val_idx]
    y_train = y.iloc[train_idx]
    y_val   = y.iloc[val_idx]

    return X_train, X_val, y_train, y_val

def get_fold_data_normalized(X, y, train_idx, val_idx):
   
    X_train = X.iloc[train_idx]
    X_val   = X.iloc[val_idx]
    y_train = y.iloc[train_idx]
    y_val   = y.iloc[val_idx]

    mean = X_train.mean(axis=0)
    std  = X_train.std(axis=0)

    y_train_mean = y_train.mean()

    X_train_norm = (X_train - mean) / std
    X_val_norm   = (X_val   - mean) / std
    y_train = y_train - y_train_mean
    y_val   = y_val   - y_train_mean

    return X_train_norm, X_val_norm, y_train, y_val

# Tensor conversion

def torch_tensor_conversion(X_train, y_train, X_val, y_val):

    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values.reshape(-1, 1), dtype=torch.float32).view(-1, 1)
    X_val_tensor   = torch.tensor(X_val.values, dtype=torch.float32)
    y_val_tensor   = torch.tensor(y_val.values.reshape(-1, 1), dtype=torch.float32).view(-1, 1)

    return X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor

def get_model(input_dim, hidden_dim, output_dim):
    ### BEGIN SOLUTION
    return torch.nn.Sequential(
        torch.nn.Linear(in_features=input_dim, out_features=hidden_dim, bias=True),     # Input layer
        torch.nn.Tanh(),                                                                # Activation function
        torch.nn.Linear(in_features=hidden_dim, out_features=output_dim, bias=True),    # Output layer
    )
    ### END SOLUTION

## 2 layer cross validation

In [ ]:
# Parameters:

outer_folds_k_1 = 10
inner_folds_k_2 = 10
random_state = 42

# ANN parameters
input_dim  = M # M number of features
output_dim = 1 # regression problem
lr = 1e-3
n_epochs = 1000
momentum = 0.9
hyperparameters_ANN = [1, 2, 3, 4, 5, 6, 7]


# Regularization parameters for linear regression
lambdas__for_linear_regression = np.logspace(-4, 3, 30)[20:27]



In [234]:
CV_outer = KFold(n_splits=outer_folds_k_1, shuffle=True, random_state=random_state) 

fold_results = {}   # Outer fold dict (key: outer fold index)
best_hyperparameters_per_fold = {}
best_lambda_per_fold = {}
outer_fold_index = 0

for outer_train_idx, outer_test_idx in CV_outer.split(X):
    outer_fold_index += 1
    X_train_outer, X_test_outer, y_train_outer, y_test_outer = get_fold_data(X, y, outer_train_idx, outer_test_idx)

    CV_inner = KFold(n_splits=inner_folds_k_2, shuffle=True, random_state=random_state)
    inner_mse_baseline = {}   # Inner fold dict (key: inner fold index)
    inner_mse_ANN = {}
    inner_mse_linear_regression = {}
    inner_fold_index = 0

    for inner_train_idx, inner_test_idx in CV_inner.split(X_train_outer):
        inner_fold_index += 1
        print(f"Outer Fold {outer_fold_index} - Inner Fold {inner_fold_index}")

        ############################# DATA Inner Fold ####################################
        X_train_inner_norm, X_test_inner_norm, y_train_inner_norm, y_test_inner_norm = get_fold_data_normalized(X_train_outer, y_train_outer, inner_train_idx, inner_test_idx)

        ############################# Linear Regression Inner Fold ####################################
        
        # Set up a dictionary to store the results for each lambda setting
        results_inner_linear_regression = {lam: {'train': [], 'test': []} for lam in lambdas__for_linear_regression}

        for lam in lambdas__for_linear_regression:

            model = Ridge(alpha=lam, random_state=42)
            model.fit(X_train_inner_norm, y_train_inner_norm)

            y_test_pred_inner = model.predict(X_test_inner_norm)
            mse_test = mean_squared_error(y_test_inner_norm, y_test_pred_inner)

            results_inner_linear_regression[lam]['test'].append(mse_test)
            inner_mse_linear_regression[inner_fold_index] = results_inner_linear_regression

        ############################# ANN Inner Fold ########################################
        X_train_inner_tensor, y_train_inner_tensor, X_test_inner_tensor, y_test_inner_tensor = torch_tensor_conversion(X_train_inner_norm, y_train_inner_norm, X_test_inner_norm, y_test_inner_norm) 
    
        # Set up a dictionary to store the results for each hyperparameter setting
        results_inner_ANN = {hidden_dim: {'train': [], 'test': []} for hidden_dim in hyperparameters_ANN}

        for hidden_dim in hyperparameters_ANN:
            # Define a model instance with a specific number of hidden units
            model = get_model(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)

            # Define loss criterion
            criterion = torch.nn.MSELoss()

            # Define the optimizer as the Adam optimizer (not needed to know the details)
            optimizer = torch.optim.SGD(params=model.parameters(), lr=lr, momentum=momentum)

            for epoch in range(n_epochs):

                # Set the model to training mode
                model.train()

                # Make a forward pass through the model to compute the outputs
                outputs = model(X_train_inner_tensor)
                # Compute the loss
                loss = criterion(outputs, y_train_inner_tensor)

                # Make sure that the gradients are zero before you use backpropagation
                optimizer.zero_grad()
                # Do a backward pass to compute the gradients wrt. model parameters using backpropagation.
                loss.backward()
                # Update the model parameters by making the optimizer take a gradient descent step
                optimizer.step()
                
                # Store the training loss for this epoch in the dictionary
                #results_inner_ANN[hidden_dim]['train'].append(loss.item())

            # Compute the final test loss on the test set
            with torch.no_grad(): # No need to compute gradients for the validation set
                model.eval()
                val_outputs = model(X_test_inner_tensor)
                val_loss = criterion(val_outputs, y_test_inner_tensor)
                results_inner_ANN[hidden_dim]['test'].append(val_loss.item())
                #print(f'  Hidden units: {hidden_dim}, Validation set MSE: {val_loss.item():.4f}')
                inner_mse_ANN[inner_fold_index] = results_inner_ANN 

        ############################# BASELINE Inner Fold ####################################
        
        y_train_mean = y_train_inner_norm.mean()
        y_test_pred_inner = pd.Series(y_train_mean, index=y_test_inner_norm.index)
        mse = mean_squared_error(y_test_inner_norm, y_test_pred_inner)
        inner_mse_baseline[inner_fold_index] = mse   # Store by fold index


    ############################ OUTER FOLD ##########################################################

    ############################ Data ##########################################################

    X_train_outer_norm, X_test_outer_norm, y_train_outer_norm, y_test_outer_norm = get_fold_data_normalized(X, y, outer_train_idx, outer_test_idx)

    X_train_outer_tensor, y_train_outer_tensor, X_test_outer_tensor, y_test_outer_tensor = torch_tensor_conversion(X_train_outer_norm, y_train_outer_norm, X_test_outer_norm, y_test_outer_norm)

    ############################ Linear Regression Outer Fold ####################################

    avg_mse_per_lambda = {}
    for lam in lambdas__for_linear_regression:
        mse_values = []
        for inner_fold in inner_mse_linear_regression.keys():
            mse_values.append(inner_mse_linear_regression[inner_fold][lam]['test'][0])  # We only have one value of test per fold 
        avg_mse = np.mean(mse_values)
        avg_mse_per_lambda[lam] = avg_mse
    
    best_lambda = min(avg_mse_per_lambda, key=avg_mse_per_lambda.get)
    best_lambda_per_fold[outer_fold_index] = best_lambda

    model = Ridge(alpha=best_lambda, random_state=42)
    model.fit(X_train_outer_norm, y_train_outer_norm)
    y_test_pred_outer = model.predict(X_test_outer_norm)
    mse_test_outer = mean_squared_error(y_test_outer_norm, y_test_pred_outer)
    print(f"For outer fold {outer_fold_index} Best λ (alpha): {best_lambda}, Test MSE: {mse_test_outer}")

    ############################ ANN Outer Fold ####################################
    # Find the best hyperparameter based on inner folds
    avg_mse_per_hyperparam = {}
    for hidden_dim in hyperparameters_ANN:
        mse_values = []
        for inner_fold in inner_mse_ANN.keys():
            mse_values.append(inner_mse_ANN[inner_fold][hidden_dim]['test'][0])  # We only have one value of test per fold 
        avg_mse = np.mean(mse_values)
        avg_mse_per_hyperparam[hidden_dim] = avg_mse
    
    best_hyperparam = min(avg_mse_per_hyperparam, key=avg_mse_per_hyperparam.get)
    best_hyperparameters_per_fold[outer_fold_index] = best_hyperparam

    model = get_model(input_dim=input_dim, hidden_dim=best_hyperparam, output_dim=output_dim)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(params=model.parameters(), lr=lr, momentum=momentum)
    model.train()
    outputs = model(X_train_outer_tensor)
    loss = criterion(outputs, y_train_outer_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    with torch.no_grad(): # No need to compute gradients for the validation set
            model.eval()
            val_outputs = model(X_test_outer_tensor)
            val_loss = criterion(val_outputs, y_test_outer_tensor)
            print(f'For outer fold {outer_fold_index} Best hidden units: {best_hyperparam}, Test MSE: {val_loss.item():.4f}')
    
    ############################ BASELINE Outer Fold ###############################
    inner_mse_mean = np.mean(list(inner_mse_baseline.values()))
    fold_results[outer_fold_index] = inner_mse_baseline   # Store the whole dict per outer fold
    print(f"For outer fold {outer_fold_index} Mean Inner fold MSE for Baseline:", inner_mse_mean)

    ############################ Linear Regression Outer Fold ############################


Outer Fold 1 - Inner Fold 1
Outer Fold 1 - Inner Fold 2
Outer Fold 1 - Inner Fold 3
Outer Fold 1 - Inner Fold 4
Outer Fold 1 - Inner Fold 5
Outer Fold 1 - Inner Fold 6
Outer Fold 1 - Inner Fold 7
Outer Fold 1 - Inner Fold 8
Outer Fold 1 - Inner Fold 9
Outer Fold 1 - Inner Fold 10
For outer fold 1 Best λ (alpha): 20.43359717856944, Test MSE: 4589.20877602697
For outer fold 1 Best hidden units: 4, Test MSE: 5545.3032
For outer fold 1 Mean Inner fold MSE for Baseline: 6068.262610662071
Outer Fold 2 - Inner Fold 1
Outer Fold 2 - Inner Fold 2
Outer Fold 2 - Inner Fold 3
Outer Fold 2 - Inner Fold 4
Outer Fold 2 - Inner Fold 5
Outer Fold 2 - Inner Fold 6
Outer Fold 2 - Inner Fold 7
Outer Fold 2 - Inner Fold 8
Outer Fold 2 - Inner Fold 9
Outer Fold 2 - Inner Fold 10
For outer fold 2 Best λ (alpha): 35.622478902624444, Test MSE: 3954.948323794025
For outer fold 2 Best hidden units: 3, Test MSE: 5586.7300
For outer fold 2 Mean Inner fold MSE for Baseline: 6064.930475785973
Outer Fold 3 - Inner F

{1: 5, 2: 3, 3: 4, 4: 3, 5: 5, 6: 3, 7: 3, 8: 4, 9: 4, 10: 4}